#### **DATA CLEANING: THE NUMBERS(TNDB)**
The Numbers' production budget data is crucial for calculating true return on investment (ROI), which is often more relevant than gross revenue alone. This enables YE Studios to identify which genres and budget ranges offer the best profit margins, not just the highest revenue. The budget-to-revenue ratio analysis helps determine the minimum viable investment for different film categories.

In [1]:
#LOAD DATA:
import pandas as pd
import numpy as np
import gzip

tndb = pd.read_csv("../data/zippedData/tn.movie_budgets.csv.gz", compression = "gzip")
tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [2]:
#CLEAN DATA:
##CLEAN COLUMN NAMES:Make column names lowercase and replace spaces with underscores, and remove parantheses.
tndb.columns = tndb.columns.str.strip().str.lower().str.replace(" ","_").str.replace("(","").str.replace(")","")
tndb.head()
tndb.dtypes

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

In [3]:
#CLEAN DATES:
#Convert release date to datetime formart.
tndb["release_date"]= pd.to_datetime(tndb["release_date"], errors = "coerce")

#Extract year for analysis:
tndb["release_year"]= tndb["release_date"].dt.year
tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017


In [4]:
#CLEAN FINACIAL DATA:
#Remove $ sign and convert to numeric.
tndb["production_budget"]= tndb["production_budget"].str.replace("$","").str.replace(",","")
tndb["production_budget"] = pd.to_numeric(tndb["production_budget"], errors="coerce")


tndb["domestic_gross"]= tndb["domestic_gross"].str.replace("$","").str.replace(",","")
tndb["domestic_gross"] = pd.to_numeric(tndb["domestic_gross"], errors="coerce")


tndb["worldwide_gross"]= tndb["worldwide_gross"].str.replace("$","").str.replace(",","")
tndb["worldwide_gross"] = pd.to_numeric(tndb["worldwide_gross"], errors="coerce")

tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017


In [5]:
#CREATE NEW COLUMNS:
#Calcuate profit
tndb["profit"]= tndb["worldwide_gross"]- tndb["production_budget"]

#Calculate return on investement(ROI)
tndb["roi"] = (tndb["profit"] / tndb["production_budget"] * 100)

#Calulate profit margin
tndb["foreign_gross"]= tndb["worldwide_gross"]- tndb["domestic_gross"]

tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,profit,roi,foreign_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,2351345279,553.257713,2015837654
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875,154.667286,804600000
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,-200237650,-57.210757,107000000
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,1072413963,324.384139,944008095
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,999721747,315.369636,696540365


In [6]:
#HANDLE DUPLICATES AND REMOVE MISSING VALUES:
#Sort by worldwide_gross and keep highest grosssing movie for each title.
tndb = tndb.sort_values("worldwide_gross", ascending= False)
tndb = tndb.drop_duplicates(subset= "movie", keep = "first")

#Drop rows where budget or worldwide_gross is missing
tndb = tndb.dropna(subset=["production_budget", "worldwide_gross"])
tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,profit,roi,foreign_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,2351345279,553.257713,2015837654
42,43,1997-12-19,Titanic,200000000,659363944,2208208395,1997,2008208395,1004.104198,1548844451
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,1747311220,571.016739,1116648995
6,7,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,2018,1748134200,582.711400,1369318718
33,34,2015-06-12,Jurassic World,215000000,652270625,1648854864,2015,1433854864,666.909239,996584239


In [7]:
##FINAL CHECK OF CLEANED DATA:
#View cleaned data
tndb.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,profit,roi,foreign_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,2351345279,553.257713,2015837654
42,43,1997-12-19,Titanic,200000000,659363944,2208208395,1997,2008208395,1004.104198,1548844451
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,1747311220,571.016739,1116648995
6,7,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,2018,1748134200,582.711400,1369318718
33,34,2015-06-12,Jurassic World,215000000,652270625,1648854864,2015,1433854864,666.909239,996584239


In [8]:
tndb.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
release_year         0
profit               0
roi                  0
foreign_gross        0
dtype: int64

In [9]:
##Saving cleaned Data:
tndb_movies_cleaned = tndb.copy()
tndb_movies_cleaned.to_csv('../data/cleanedData/tndb_cleaned_data.csv', index=False)

## Data Cleaning: TMDB Movies

The TMDB dataset provides vital metrics on movie popularity and genre classification. To ensure the data is ready for analysis, the following preprocessing steps were performed:

### 1. Removing Redundancy
The `Unnamed: 0` column was dropped, as it represents a duplicate index and does not add analytical value. Removing this column helps streamline the dataframe.

### 2. Temporal Feature Engineering
The `release_date` column was converted to a standard `datetime` format. From this, a new feature, `release_year`, was extracted to enable time-series and trend-based analysis of movie releases.

### 3. Genre Data Parsing
The `genre_ids` column is stored as a string representation of a list (e.g., `"[12, 14]"`). The `ast.literal_eval` function was used to convert these strings into actual Python lists. This transformation allows the genre IDs to be mapped to their corresponding genre names (such as *Action* or *Comedy*) in later stages of the analysis.


In [10]:
# ==========================================
# DATA CLEANING: TMDB Movies
# ==========================================

import ast
import pandas as pd
# Load the dataset directly using the relative path
tmdb_movies = pd.read_csv('../data/zippedData/tmdb.movies.csv.gz')
# Preview the first few rows to ensure successful loading
tmdb_movies.head()

# 1. Drop redundant index column
if 'Unnamed: 0' in tmdb_movies.columns:
    tmdb_movies.drop(columns=['Unnamed: 0'], inplace=True)

# 2. Feature Engineering: Convert release_date to Datetime and extract Year
tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'])
tmdb_movies['release_year'] = tmdb_movies['release_date'].dt.year

# 3. Clean Genre IDs 
# They are strings like "[12, 14]". We turn them into actual Python lists.
tmdb_movies['genre_ids'] = tmdb_movies['genre_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Data Export

After completing feature engineering and data cleaning, the processed TMDB dataset was saved as a CSV file. This ensures the cleaned data is readily available for the exploratory data analysis (EDA) phase, eliminating the need to rerun the preprocessing and cleaning steps.


In [11]:
tmdb_movies_cleaned = tmdb_movies.copy()
tmdb_movies_cleaned.to_csv('../data/cleanedData/tmdb_cleaned_data.csv', index=False)

# IMDB DATASET

#### Introduction 
The cleaned IMDB dataset provides standardized information on movie titles, genres, runtime, release year, ratings, and vote counts, prepared for analysis. The cleaning process involved merging relevant tables, handling missing values, correcting data types, and removing duplicates to improve data quality and consistency.

This refined dataset supports reliable analysis of movie characteristics and audience reception and will be used alongside other sources to identify trends and inform recommendations on the types of films the studio should produce.

#### Import libraries

In [12]:
import pandas as pd
import sqlite3

#### PATH 

In [13]:
data_path = '../data/zippedData/' # Set data path

#### Connect to IMDB database

In [14]:
conn = sqlite3.connect(data_path + "im.db")

What this does

Opens the IMDB SQLite database

Creates connection called conn

# LOADING IMBD DATA

The IMDB dataset consists of two separate tables: movie_basics and movie_ratings. These tables were merged using the movie_id column to create a unified dataset containing movie titles, genres, runtime, and ratings. Merging the datasets allows for comprehensive analysis of movie characteristics and performance.

#### Load first IMDB table (movie_basics)

In [15]:
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


#### Load second IMDB table (movie_ratings)

In [16]:
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


#### Merge both tables

In [17]:
imdb_movies = movie_basics.merge(movie_ratings, on="movie_id")
imdb_movies.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


# CLEANING

In [18]:
imdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 5.1+ MB


The imdb_movies.info() function was used to inspect the dataset structure and identify missing values. Columns such as runtime_minutes and genres had fewer non-null entries compared to the total number of rows, indicating missing data that required cleaning before analysis.

#### 1. Remove columns we don’t need
We don’t need original_title.

In [19]:
imdb_movies = imdb_movies.drop(columns=['original_title'])

#### 2. Handle missing runtime
runtime_minutes → 66236 non-null But total rows = 73856 So about 7,000 missing.

We cannot analyze runtime if missing.

Remove those rows:

In [20]:
imdb_movies = imdb_movies.dropna(subset=['runtime_minutes'])

The runtime_minutes column contained several missing values. Since runtime is a key variable in analyzing movie performance and the dataset remained sufficiently large after removal, rows with missing runtime values were dropped to ensure accuracy and consistency in analysis.

#### 3. Remove missing genres

In [21]:
imdb_movies = imdb_movies.dropna(subset=['genres'])

The genres column contained missing values. Since genre is a key variable for identifying movie categories and determining performance by film type, rows with missing genre information were removed to ensure accurate analysis

#### 4. Convert runtime to integer

In [22]:
imdb_movies['runtime_minutes'] = imdb_movies['runtime_minutes'].astype(int)

The runtime_minutes column was converted from float to integer to reflect actual movie runtime values in whole minutes and ensure consistency for analysis.

#### 5. Remove duplicates

Sometimes same movie appears twice.

In [23]:
imdb_movies = imdb_movies.drop_duplicates(subset='primary_title')

6. Final cleaned dataset check

In [24]:
imdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62444 entries, 0 to 73852
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         62444 non-null  object 
 1   primary_title    62444 non-null  object 
 2   start_year       62444 non-null  int64  
 3   runtime_minutes  62444 non-null  int32  
 4   genres           62444 non-null  object 
 5   averagerating    62444 non-null  float64
 6   numvotes         62444 non-null  int64  
dtypes: float64(1), int32(1), int64(2), object(3)
memory usage: 3.6+ MB


- no missing runtime

- no missing genres

- clean dataset

#### 7. convert genres to lowercase

In [25]:
imdb_movies['genres'] = imdb_movies['genres'].str.lower()

The IMDB dataset has been cleaned by removing unnecessary columns, handling missing values and converting data types. This ensured the dataset was accurate and suitable for analysis...

In [26]:
imdb_movies_cleaned = imdb_movies.copy()
imdb_movies_cleaned.to_csv('../data/cleanedData/imdb_cleaned_data.csv', index = False)
